In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

## plt_show function

In [12]:
def plt_show(image, title=""):
    if np.ndim(image) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image, cmap='Greys_r')
    plt.show()

## FaceDetection and VideoCamera class

In [13]:
class FaceDetection(object):
    def __init__(self, xml_path):
        self.face_cascade = cv2.CascadeClassifier(xml_path)
        
    def detect(self, frame):
        gray = frame
        if np.ndim(frame) == 3:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        
        return face

In [14]:
class VideoCamera(object):
    def __init__(self, index=0):
        self.webcam = cv2.VideoCapture(index)
        self.index = index
        print(self.webcam.isOpened())
        
    def get_frame(self, gray=False):
        _, frame = self.webcam.read()
        if gray:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return frame
    
    def delete(self):
        self.webcam.release()
        cv2.destroyAllWindows()
        print(self.webcam.isOpened())

## Image Normalization
-  Cut the face
-  Normalized pixel intensity
-  Resize face image
-  Align face image

### 1. Cut the faces


In [15]:
def cut_faces(image, face_cord):
    faces = []
    
    for x, y, w, h in face_cord:
        w_rm = int(0.1 * w/2)
#         print(w_rm)
        faces.append(image[y:y+h, x+w_rm:x+w-w_rm])
    
    return faces

### 2. Normalized pixel intensity
1. convert image from rgb to gray
2. contrast enhanced

In [16]:
def normalized(images):
    images_norm = []
    
    for image in images:
        if np.ndim(image) == 3:
            images_norm.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        else:
            # contrast pixel intensity
            images_norm.append(cv2.equalizeHist(image))
    return images_norm

### 3. Resize

In [17]:
def resize(frames, size=(50, 50)):
    image_resize = []
    
    for frame in frames:
        if frame.shape < size:
            img = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
        else:
            img = cv2.resize(frame, size, interpolation=cv2.INTER_CUBIC)
        image_resize.append(img)
    
    return image_resize

### 4. Align Image

In [18]:
# pending

### Get all function together

In [19]:
def normalized_images(image, face):
    cut = cut_faces(image, face)
    norm = normalized(cut)
    res = resize(norm)
    return res;

# draw rectangle around face
def drawRect(image, face):
    for x, y, w, h in face:
        w_rmv = int(0.2 * w/2)
        cv2.rectangle(image, (x+w_rmv, y), (x+w-w_rmv, y+h), (0, 0, 0, 255), 2)

### create dataset of 30 face each person

In [21]:
from IPython.display import clear_output

cwd = '/media/rohitkumar/Rohit-Sonu/python3/projects/Face_Recognition'
folder = cwd+'/people/'+ input('Person name: ').lower()

webcam = VideoCamera(0)
detector = FaceDetection('/media/rohitkumar/Rohit-Sonu/python3/projects/Face_Recognition/haarcascade_frontalface_default.xml')
       

if not os.path.exists(folder):
    os.mkdir(folder)
    counter = 0
    timer = 0
    
     
    while counter<50:
    
        frame = webcam.get_frame()
        face_coord = detector.detect(frame)
        
        if len(face_coord) and timer%700 == 50:
            norm_frame = normalized_images(frame, face_coord)
            cv2.imwrite(folder+'/'+str(counter)+'.jpg', norm_frame[0])
            plt_show(norm_frame[0])
            clear_output(wait=True)
            counter += 1
        drawRect(frame, face_coord)
        cv2.imshow("Face", frame)
        
        cv2.waitKey(50)
        timer += 50
    webcam.delete()
else:
    print('user already exist')

False


# Training the model

_recognizer.train(image, label)_

-  images: list of numpy arrays
-  labels: numpy array with the label corresponding to image

In [22]:
def collect_dataset():
    images = []
    labels = []
    label_dic = {}
    
    persons = [person for person in os.listdir('people/')]
    
    for idx, person in enumerate(persons):
        label_dic[idx] = person
        
        for img in os.listdir('people/'+person):
            images.append(cv2.imread('people/'+person+'/'+img, 0))
            labels.append(idx)
            
    return (images, np.array(labels), label_dic)
    

In [23]:
images, labels, label_dic = collect_dataset()

# train eigen model
eig_rec = cv2.face.EigenFaceRecognizer_create()
eig_rec.train(images, labels)

# train fisher model
fish_rec = cv2.face.FisherFaceRecognizer_create()
fish_rec.train(images, labels)

# train LBPHF model
lbphf = cv2.face.LBPHFaceRecognizer_create()
lbphf.train(images, labels)

## Predict faces

-  pred, conf = recognizer.train(image)

### Prediction on still images

In [24]:
detector = FaceDetection('/media/rohitkumar/Rohit-Sonu/python3/projects/Face_Recognition/haarcascade_frontalface_default.xml')
frame = cv2.imread('../my_face.png', 0)
face_coord = detector.detect(frame)

norm_images = normalized_images(frame, face_coord)
# collector = cv2.face.MinDistancePredictCollector()
pred, conf = eig_rec.predict(norm_images[0])
print(label_dic[pred].capitalize(), conf)
# conf = collector.getDist()
# predict = collector.getLabel()

pred, conf = fish_rec.predict(norm_images[0])
print(label_dic[pred].capitalize(), conf)

pred, conf = lbphf.predict(norm_images[0])
print(label_dic[pred].capitalize(), conf)

Rohit 3027.278560467097
Rohit 159.59568745909917
Rohit 134.797004927861


### Prediction on live videos

In [27]:
webcam = VideoCamera(0)
detector = FaceDetection('/media/rohitkumar/Rohit-Sonu/python3/projects/Face_Recognition/haarcascade_frontalface_default.xml')

while True:
    frame = webcam.get_frame()
    face_coord = detector.detect(frame)
    
    if len(face_coord):
        norm_frame = normalized_images(frame, face_coord)
        thershold = 140
        for i, face in enumerate(norm_frame):
            pred, conf = lbphf.predict(face)
            
            if conf < thershold:
                cv2.putText(frame, label_dic[pred].capitalize(), (face_coord[i][0] , face_coord[i][1]-10),
                        cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 2)
            else:
                cv2.putText(frame, "Unknown".capitalize(), (face_coord[i][0] , face_coord[i][1]-10),
                        cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 2)
    drawRect(frame, face_coord)
    cv2.imshow("Face Recognizer", frame)
#     print(label_dic[pred].capitalize())
    
    if cv2.waitKey(20) & 0xff == 27:
        break;
webcam.delete()

True
False
